In [1]:
!pip install jiwer
!pip install trectools
!pip install rouge_score
!pip install datasets transformers torch evaluate nltk rouge_score

import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 1.6 MB/s eta 0:00:00
  Attempting uninstall: levenshtein
    Found existing installation: Levenshtein 0.20.9
    Uninstalling Levenshtein-0.20.9:
      Successfully uninstalled Levenshtein-0.20.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
python-levenshtein 0.20.9 requires Levenshtein==0.20.9, but you have levenshtein 0.20.2 which is incompatible.
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Created wheel for trectools: filename=trectools-0.0.49-py3-none-any.whl size=27140 sha256=dae8775ecce7258db101e9704ba7cc2a5a67a843b35b25cd4572c47d2cf25369
  Stored in directory: /root/.cache/pip/wheels/66/02/63/20592b3c87f6860ff74e8c58cffe040a7961c358ec404ceade
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=fb98aa0db245548482aa0b78c839c

In [2]:
df = pd.read_csv('/kaggle/input/corpus-of-russian-news-articles-from-lenta/lenta-ru-news.csv')
df = df[['title', 'text']]
df = df.dropna()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
import torch
import nltk
from datasets import load_dataset
from transformers import AutoConfig
nltk.download('punkt')
import string
from datasets import Dataset
from transformers import T5ForConditionalGeneration, T5Tokenizer

prefix = ""
max_input_length = 1000
max_target_length = 100

tokenizer = T5Tokenizer.from_pretrained('cointegrated/rut5-base-absum')

def cut_text(s):
    if len(s) > 1000:
        s = s[0:1000]
    return s

def cut_title(s):
    if len(s) > 100:
        s = s[0:100]
    return s

df['text'] = df['text'].apply(cut_text)
df['title'] = df['title'].apply(cut_title)

pd_ds = df[['text', 'title']]

def clean_text(text):
  sentences = nltk.sent_tokenize(text.strip())
  sentences_cleaned = [s for sent in sentences for s in sent.split("\n")]
  sentences_cleaned_no_titles = [sent for sent in sentences_cleaned
                                 if len(sent) > 0 and
                                 sent[-1] in string.punctuation]
  text_cleaned = "\n".join(sentences_cleaned_no_titles)
  return text_cleaned

def tokinize_func(examples):
  texts_cleaned = [clean_text(text) for text in examples["text"]]
  inputs = [prefix + text for text in texts_cleaned]
  model_inputs = tokenizer(inputs, padding='max_length', max_length=max_target_length, truncation=True)

  # Setup the tokenizer for targets
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples["title"], padding='max_length', max_length=max_target_length, truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs


dataset = Dataset.from_pandas(pd_ds)
dataset = dataset.select(range(100_000))

test_part = 0.02
dataset = dataset.select(range(int(len(dataset)*(1-test_part)), int(len(dataset))))

dataset = dataset.map(tokinize_func, batched=True)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Downloading:   0%|          | 0.00/808k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/315 [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [4]:
from evaluate import evaluator
from transformers import pipeline
from datasets import load_dataset
from transformers import pipeline, AutoModel

task_evaluator = evaluator("text2text-generation")

model = T5ForConditionalGeneration.from_pretrained("/kaggle/input/t5-title-gen-fine-tuned/results/ruT5_title_generator")
generator = pipeline(task="text2text-generation", model=model, tokenizer=tokenizer)

def eval_model(generator, metric='rouge'):
    results = task_evaluator.compute(
        model_or_pipeline=generator,
        data=dataset,
        input_column="text",
        label_column="title",
        metric=metric,
    )
    print(results)

In [5]:
eval_model(generator)

2023-01-25 09:08:14.412315: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'rouge1': 0.07278906926406926, 'rouge2': 0.009727777777777778, 'rougeL': 0.07197240259740262, 'rougeLsum': 0.07247694805194807, 'total_time_in_seconds': 2472.747589603, 'samples_per_second': 0.8088168838620121, 'latency_in_seconds': 1.2363737948015}


In [6]:
def summarize(
    text, n_words=None, compression=None,
    max_length=1000, num_beams=3, do_sample=False, repetition_penalty=10.0, 
    **kwargs
            ):
    if n_words:
        text = '[{}] '.format(n_words) + text
    elif compression:
        text = '[{0:.1g}] '.format(compression) + text
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    with torch.inference_mode():
        out = model.generate(
            **x, 
            max_length=max_length, num_beams=num_beams, 
            do_sample=do_sample, repetition_penalty=repetition_penalty, 
            **kwargs
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

def predict(i):
    text = dataset[i]['text']
    title = dataset[i]['title']
    predicted = summarize(text)
    
    print('Новость: ', text, '\n')
    print('Оригинальное название: ', title, '\n')
    print('Сгенерированное название: ', predicted, '\n')

In [7]:
# predicting title for news №5
predict(5)

Новость:  Объединенные Арабские Эмираты в лице президента страны шейха Заеда бин Султана ан-Нихайяна приняли решение отозвать пожертвование на сумму 2,5 миллиона долларов Университетской богословской школе Гарварда, сообщает  Reuters. Пожертвование богословской школе было сделано в 2000 году для поощрения исследований исламской религии. Однако недавно на сайте Гарварда появился пресс-релиз, в котором сообщалось, что университет рассматривает вопрос о возвращении шейху его денег в связи со скандалом вокруг Международного центра сотрудничества и развития имени Заеда. Ранее несколько сот студентов и выпускников подписали петицию в поддержку этой инициативы. Центр имени Заеда, основанный в 1999 году, привлек внимание к своей деятельности после того, как приглашенные арабские и европейские специалисты прочитали ряд лекций, посвященных Израилю и иудаизму. В антисемитизме были, в частности, обвинены сотрудничавшие с центром французский писатель и один из экспертов. По мнению первого, трагедия

In [8]:
# predicting title for news №50
predict(50)

Новость:  В четверг на территории США успешно завершилось первое испытание израильско-американской системы ПВО "Хец" (Arrow) по перехвату реальной цели, сообщает агентство "Курсор". Новая ракета поразила баллистическую ракету "Скад", запущенную с корабля у побережья Калифорнии. "Хец", поднявшаяся в воздух с острова на расстоянии нескольких километров, опознала цель и уничтожила ее. Это первое испытание новой системы ПВО в "боевых условиях", отмечает агентство. Ранее в качестве целей использовались реактивные снаряды, имитирующие баллистические ракеты или компьютерные имитаторы. Система ПВО "Хец" является плодом сотрудничества израильской государственной корпорации "Авиационная промышленность" и американской корпорации "Боинг". Работа на проектом "противоракеты" началась еще в 80-х годах прошлого века. На сегодняшний день эта система ПВО является одной из немногих, которые способны перехватывать баллистические ракеты на больших высотах. 

Оригинальное название:  США и Израиль провели ис